### Q1. Install and load the latest versions of TensorFlow and Keras. Print their versions. 
# Q2. Load the Wine Quality dataset and explore its dimensions.Dataset link:https://www.kaggle.com/datasets/nareshbhat/wine-quality-binary-classification
# Q3. Check for null values, identify categorical variables, and encode them.
# Q4. Separate the features and target variables from the dataframe.
# Q5. Perform a train-test split and divide the data into training, validation, and test datasets.
# Q6. Perform scaling on the dataset.
# Q7. Create at least 2 hidden layers and an output layer for the binary categorical variables.
# Q8. Create a Sequential model and add all the layers to it.
# Q9. Implement a TensorBoard callback to visualize and monitor the model's training process.
# Q10. Use Early Stopping to prevent overfitting by monitoring a chosen metric and stopping the training if
no improvement is observed.
# Q11. Implement a ModelCheckpoint callback to save the best model based on a chosen metric during
training.
# Q12. Print the model summary.
# Q13. Use binary cross-entropy as the loss function, Adam optimizer, and include the metric ['accuracy'].
# Q14. Compile the model with the specified loss function, optimizer, and metrics.
# Q15. Fit the model to the data, incorporating the TensorBoard, Early Stopping, and ModelCheckpoint
callbacks.
# Q16. Get the model's parameters.
# Q17. Store the model's training history as a Pandas DataFrame.
# Q18. Plot the model's training history.
# Q19. Evaluate the model's performance using the test data.

import tensorflow as tf

In [51]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [52]:
import pandas as pd
from keras.layers import Dense,BatchNormalization,Activation,Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [53]:
df=pd.read_csv('wine/winequality-red.csv')

In [54]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [56]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [57]:
from sklearn.model_selection import train_test_split
x=df.drop('quality',axis=1)
y=df['quality']

In [58]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [59]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)


# Tensorboard callback

In [61]:
import os
from keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint

In [62]:
log_dir=os.path.join('logs','fit',pd.Timestamp.now().strftime("%Y%m%d-%H%M%S"))

tensorbord=TensorBoard(log_dir=log_dir,histogram_freq=1)

# Early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# Model checkpoint callback
checkpoint_path = "checkpoint.weights.h5"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)


In [63]:
x_train.shape[1]

11

In [64]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(11,)),
    Dense(32, activation='relu'),
    Dense(1,activation='linear')  # Output layer with 1 neuron for regression
])

C:\Users\www58\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [65]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[tensorbord,model_checkpoint,early_stopping_callback]
)

Epoch 1/100


In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_df['accuracy'], label='Train Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.plot(history_df['loss'], label='Train Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy / Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
